In [1]:
import os
import uuid
import numpy as np
import pandas as pd
import itertools

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from isadream import demos, demo_path, io
from isadream.models import utils

In [14]:
x_groups = (
    ('Al-Na Distance', ('Al-M Distance',), ("Al",)),
    ('Periodic Box Size', ('Periodic Box Size',), ("Al")),
    ('Solvent Molecule Count', ('Solvent Molecule Count',), ("Al")),
)

y_groups = (
    ('Isotropic Shielding Factor', ('Isotropic Shielding Factor',), ("Al")),
)

test_groups = x_groups + y_groups

In [15]:
demo_json = io.read_idream_json(os.path.join(demo_path, demos["ERNESTO_NMR_1"]))
node = io.parse_node_json(demo_json)

In [16]:
main_df, metadata_df, metadata = io.prepare_nodes_for_bokeh(x_groups, y_groups, [node])

In [18]:
main_df

,Al-Na Distance,Periodic Box Size,Solvent Molecule Count,Isotropic Shielding Factor
0,4.0,14.35,90,535.5807
1,4.0,14.35,90,539.3779
2,4.0,14.35,90,531.1274
3,4.0,14.35,90,527.5177
4,4.0,14.35,90,534.3948
5,4.0,14.35,90,524.4831
6,4.0,14.35,90,524.0155
7,4.0,14.35,90,533.2804
8,4.0,14.35,90,527.3082
9,4.0,14.35,90,521.0975


In [4]:
x_groups=(('Total Aluminate Concentration', ('Molar',), ("Al",)),
          ('Counter Ion Concentration', ('Molar',),
           ("Na+", "Li+", "Cs+", "K+")),
          ('Counter Ion', ('Species',), ("Na+", "Li+", "Cs+", "K+",)),
          ('Base Concentration', ('Molar',), ("OH-",)))

y_groups=(('27 Al ppm', ('ppm',), ("Al",)),)

test_groups = x_groups + y_groups

In [5]:
demo_json = io.read_idream_json(os.path.join(demo_path, demos["SIPOS_NMR_V2"]))
node = io.parse_node_json(demo_json)

In [6]:
json_paths = [os.path.join(demo_path, demo)
              for demo in [demos["SIPOS_NMR"], demos["SIPOS_NMR_V2"]]]

nodes = io.create_drupal_nodes(json_paths)
main_df, metadata_df, metadata = io.prepare_nodes_for_bokeh(x_groups, y_groups, nodes)

In [7]:
main_df

,Total Aluminate Concentration,Counter Ion Concentration,Counter Ion,Base Concentration,27 Al ppm
0,0.005,2.93,K+,2.93,79.90
1,0.005,4.92,K+,4.92,79.84
2,0.005,6.85,K+,6.85,79.72
3,0.005,9.13,K+,9.13,79.66
4,0.005,10.71,K+,10.71,79.66
5,0.005,0.66,Li+,0.66,79.92
6,0.005,1.10,Li+,1.10,79.84
7,0.005,1.64,Li+,1.64,79.66
8,0.005,2.14,Li+,2.14,79.54
9,0.005,2.59,Li+,2.59,79.36


In [8]:
data, metadata = io.collate_node(node, test_groups)
# display(data)
# type(data)
# display(data[0])
# data[("Counter Ion Concentration")]
# data.iloc[[0, 1], :]
data
# data.swaplevel(0,1, axis=1).xs("data", axis=1)
# data.swaplevel(0,1, axis=1).xs("metadata", axis=1).iloc[[0, 1], :]

Total Aluminate Concentration  \
                            data   
0                          0.050   
1                          0.050   
2                          0.050   
3                          0.050   
4                          0.050   
5                          0.050   
6                          0.050   
7                          0.050   
8                          0.050   
9                          0.050   
10                         0.050   
11                         0.100   
12                         0.100   
13                         0.100   
14                         0.100   
15                         0.100   
16                         0.100   
17                         0.100   
18                         0.100   
19                         0.100   
20                         0.100   
21                         0.100   
22                         0.396   
23                         0.396   
24                         0.396   
25                         0.396   
26                         0.396   
27                         0.396   
28                         0.396   
29                         0.396   
..                           ...   
39                         0.800   
40                         0.800   
41                         0.800   
42                         1.200   
43                         1.200   
44                         1.200   
45                         1.200   
46                         1.200   
47                         1.200   
48                         1.200   
49                         1.200   
50                         1.200   
51                         1.200   
52                         1.200   
53                         2.000   
54                         2.000   
55                         2.000   
56                         2.000   
57                         2.000   
58                         2.000   
59                         2.000   
60                         2.000   
61                         2.000   
62                         3.000   
63                         3.000   
64                         3.000   
65                         3.000   
66                         3.000   
67                         3.000   
68                         3.000   

                                                       \
                                             metadata   
0   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
1   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
2   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
3   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
4   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
5   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
6   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
7   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
8   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
9   (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
10  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
11  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
12  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
13  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
14  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
15  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
16  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
17  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
18  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
19  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
20  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
21  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
22  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
23  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
24  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
25  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
26  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
27  (91746262-7cd3-37ac-b6c2-0dee2e95ea21, 14d04e6...   
28  (91746262-7cd3-37ac-